In [ ]:
!pip install datasets transformers accelerate peft bitsandbytes trl evaluate rouge_score bert_score pyarrow
!pip install wandb matplotlib seaborn

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f0689f0fa58cc18c355ae6e71a96d6d2cd297936b41dfd4518b96bae1c1621cb
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
import torch
from datasets import load_dataset, Dataset, load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import json
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import evaluate

In [ ]:
gdrive_path = "/content/drive/My Drive/LLM-FineTuning"
test_dataset_gdrive_path = "/content/drive/My Drive/LLM-FineTuning/test_dataset"
train_dataset_gdrive_path = "/content/drive/My Drive/LLM-FineTuning/train_dataset"
checkpoint_gdrive_path = "/content/drive/My Drive/LLM-FineTuning/tinyllama-medical-assistant"
final_model_gdrive_path = "/content/drive/My Drive/LLM-FineTuning/tinyllama-medical-assistant-final"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

Using device: cuda
GPU: Tesla T4


In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
TRAIN_SIZE = 15407
TEST_SIZE = 1000

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

In [ ]:
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset", split="train")
print(f"Total examples: {len(dataset)}")

dataset_shuffled = dataset.shuffle(seed=42)
train_raw = dataset_shuffled.select(range(TRAIN_SIZE))
test_raw = dataset_shuffled.select(range(TRAIN_SIZE, TRAIN_SIZE + TEST_SIZE))

print(f"Training examples: {len(train_raw)}")
print(f"Test examples: {len(test_raw)}")

README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

Total examples: 16407
Training examples: 15407
Test examples: 1000


In [ ]:
def format(example):
    """Format the MedQuAD data into chat format"""

    # Get question type and question and answer from the dataset
    # Check different possible column names

    # we skip question types for better results
    # question_type = (example.get('qtype') or '')

    question = (example.get('question') or
                example.get('Question') or
                example.get('qtext') or
                example.get('query') or '')

    answer = (example.get('answer') or
              example.get('Answer') or
              example.get('atext') or
              example.get('response') or '')

    # Clean the text
    # question_type = question_type.strip()
    question = question.strip()
    answer = answer.strip()

    # Skip if either question or answer is missing
    if not question or not answer:
        return None

    # Create medical-focused chat format
    chat = [
        {"role": "system", "content": """You are Amino AI, a knowledgeable medical assistant designed to provide helpful, accurate health information.

Your guidelines:
- Provide clear, evidence-based medical information
- Use simple language that patients can understand
- Always emphasize that your advice doesn't replace professional medical care
- For serious symptoms or concerns, strongly recommend seeing a healthcare provider
- Be empathetic and supportive in your responses
- If uncertain about medical information, acknowledge limitations
- Never diagnose specific conditions or prescribe medications

Remember: You're here to educate and guide, not to replace doctors."""},
        {"role": "user", "content": question},
        {"role": "assistant", "content": answer}
    ]

    # Apply chat template
    formatted_text = tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=False
    )

    return {"text": formatted_text}

In [ ]:
print("Processing training data...")
processed_train_data = []
for example in tqdm(train_raw):
    formatted = format(example)
    if formatted:
        processed_train_data.append(formatted)
print(f"Successfully processed {len(processed_train_data)} training examples")

Processing training data...


100%|██████████| 15407/15407 [00:02<00:00, 6403.70it/s]

Successfully processed 15407 training examples


In [ ]:
print("Processing test data...")
processed_test_data = []
for example in tqdm(test_raw):
    formatted = format(example)
    if formatted:
        processed_test_data.append(formatted)
print(f"Successfully processed {len(processed_test_data)} test examples")

Processing test data...


100%|██████████| 1000/1000 [00:00<00:00, 3438.57it/s]

Successfully processed 1000 test examples


In [ ]:
test_dataset = Dataset.from_list(processed_test_data)
test_dataset.save_to_disk(test_dataset_gdrive_path)

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_dataset = Dataset.from_list(processed_train_data)
train_dataset.save_to_disk(train_dataset_gdrive_path)

Saving the dataset (0/1 shards):   0%|          | 0/15407 [00:00<?, ? examples/s]

In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors=None
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

Map:   0%|          | 0/15407 [00:00<?, ? examples/s]

In [ ]:
train_size = int(0.9 * len(tokenized_train_dataset))
eval_size = len(tokenized_train_dataset) - train_size
final_train_dataset, eval_dataset = torch.utils.data.random_split(
    tokenized_train_dataset, [train_size, eval_size]
)
print(f"Final train size: {len(final_train_dataset)}, Eval size: {len(eval_dataset)}")

Final train size: 13866, Eval size: 1541


In [ ]:
def plot_dataset_stats():
    """Plot dataset statistics"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Text length distribution
    text_lengths = [len(item['text']) for item in processed_train_data]
    axes[0].hist(text_lengths, bins=50, alpha=0.7, color='lightcoral')
    axes[0].set_title('Text Length Distribution')
    axes[0].set_xlabel('Text Length (characters)')
    axes[0].set_ylabel('Frequency')

    # Dataset size comparison
    dataset_names = ['Original Dataset', 'Training Set', 'Test Set', 'Validation Set']
    dataset_sizes = [len(dataset), len(processed_train_data), len(processed_test_data), len(eval_dataset)]

    axes[1].bar(dataset_names, dataset_sizes, color=['purple', 'orange', 'green', 'red'])
    axes[1].set_title('Dataset Sizes Comparison')
    axes[1].set_ylabel('Number of Examples')
    axes[1].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.savefig(gdrive_path + '/dataset_statistics.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_dataset_stats()

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,  # Rank
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

training_args = TrainingArguments(
    output_dir=checkpoint_gdrive_path,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    warmup_steps=100,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    eval_steps=50,
    eval_strategy="steps",
    save_steps=100,
    save_total_limit=3,
    push_to_hub=False,
    report_to="none",
    dataloader_pin_memory=False,
    logging_dir="./logs",
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 12,615,680 || all params: 1,112,664,064 || trainable%: 1.1338


In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

In [ ]:
# resume_from_checkpoint=True
trainer.train()

In [ ]:
trainer.save_model(final_model_gdrive_path)
tokenizer.save_pretrained(final_model_gdrive_path)

In [ ]:
def plot_training_history(trainer):
    """
    Plots comprehensive training and evaluation metrics from the trainer's log history.
    """
    # Get the log history from the trainer state
    log_history = trainer.state.log_history

    # Separate logs for training, evaluation, and learning rate
    training_logs = [log for log in log_history if 'loss' in log]
    eval_logs = [log for log in log_history if 'eval_loss' in log]

    # Extract steps and values
    train_steps = [log['step'] for log in training_logs]
    train_losses = [log['loss'] for log in training_logs]

    eval_steps = [log['step'] for log in eval_logs]
    eval_losses = [log['eval_loss'] for log in eval_logs]

    # Learning rate is usually logged with training loss
    learning_rates = [log['learning_rate'] for log in training_logs if 'learning_rate' in log]
    lr_steps = [log['step'] for log in training_logs if 'learning_rate' in log]


    # Create a 2x2 subplot
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Training and Evaluation Metrics', fontsize=16, fontweight='bold')

    # 1. Training Loss
    if train_losses:
        axes[0, 0].plot(train_steps, train_losses, 'b-', linewidth=2, label='Training Loss')
        axes[0, 0].set_title('Training Loss Over Time', fontsize=14)
        axes[0, 0].set_xlabel('Steps')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].legend()

    # 2. Validation Loss
    if eval_losses:
        axes[0, 1].plot(eval_steps, eval_losses, 'r-', linewidth=2, label='Validation Loss')
        axes[0, 1].set_title('Validation Loss Over Time', fontsize=14)
        axes[0, 1].set_xlabel('Steps')
        axes[0, 1].set_ylabel('Loss')
        axes[0, 1].grid(True, alpha=0.3)
        axes[0, 1].legend()

    # 3. Learning Rate Schedule
    if learning_rates:
        axes[1, 0].plot(lr_steps, learning_rates, 'g-', linewidth=2)
        axes[1, 0].set_title('Learning Rate Schedule', fontsize=14)
        axes[1, 0].set_xlabel('Steps')
        axes[1, 0].set_ylabel('Learning Rate')
        axes[1, 0].grid(True, alpha=0.3)
        axes[1, 0].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

    # 4. Combined Training vs Validation Loss
    if train_losses and eval_losses:
        # Interpolate training loss to match the evaluation steps for a direct comparison
        train_loss_interp = np.interp(eval_steps, train_steps, train_losses)

        axes[1, 1].plot(eval_steps, train_loss_interp, 'b-', linewidth=2, label='Training Loss (interpolated)')
        axes[1, 1].plot(eval_steps, eval_losses, 'r-', linewidth=2, label='Validation Loss')
        axes[1, 1].set_title('Training vs Validation Loss', fontsize=14)
        axes[1, 1].set_xlabel('Steps')
        axes[1, 1].set_ylabel('Loss')
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to make room for suptitle
    plt.savefig(gdrive_path + '/training_metrics.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
plot_training_history(trainer)

In [ ]:
print("="*50)
print("LOADING PRE-TRAINED MODEL FROM GOOGLE DRIVE")
print("="*50)

print("1. Load the Base Model")
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

print("2. Load PEFT Model (Apply Fine-Tuned Adapters)")
model = PeftModel.from_pretrained(base_model, final_model_gdrive_path)

print("3. Merge Adapters for Faster Inference")
model = model.merge_and_unload()

print("4. Load the Tokenizer")
tokenizer = AutoTokenizer.from_pretrained(final_model_gdrive_path)
tokenizer.pad_token = tokenizer.eos_token

print("="*50)
print("MODEL AND TOKENIZER LOADED SUCCESSFULLY.")
print("="*50)

LOADING PRE-TRAINED MODEL FROM GOOGLE DRIVE
1. Load the Base Model
2. Load PEFT Model (Apply Fine-Tuned Adapters)
3. Merge Adapters for Faster Inference
4. Load the Tokenizer
MODEL AND TOKENIZER LOADED SUCCESSFULLY.


In [ ]:
def chat(question):
    chat = [
        {"role": "system", "content": """You are Amino AI, a knowledgeable medical assistant designed to provide helpful, accurate health information.

Your guidelines:
- Provide clear, evidence-based medical information
- Use simple language that patients can understand
- Always emphasize that your advice doesn't replace professional medical care
- For serious symptoms or concerns, strongly recommend seeing a healthcare provider
- Be empathetic and supportive in your responses
- If uncertain about medical information, acknowledge limitations
- Never diagnose specific conditions or prescribe medications

Remember: You're here to educate and guide, not to replace doctors."""},
        {"role": "user", "content": question}
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Store the length of the input tokens
    input_token_length = inputs.shape[1]

    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=512, # used to be 200
            do_sample=True,
            temperature=0.7, # can be 0.6 to be more strict on answers
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )

    # Get the token IDs that are ONLY from the generation
    generated_token_ids = outputs[0, input_token_length:]

    # Decode only the newly generated tokens
    response = tokenizer.decode(generated_token_ids, skip_special_tokens=True)

    return response.strip()

In [ ]:
test_cases = [
    "What are the symptoms of diabetes?",
    "How can I prevent heart disease?",
    "What causes high blood pressure?",
    "What are the side effects of aspirin?",
    "How is pneumonia treated?",
    "What are the signs of a heart attack?",
    "How can I improve my immune system?",
    "What causes headaches?"
]

test_results = []
for i, question in enumerate(test_cases):
    print(f"\n🏥 Medical Question {i+1}: {question}")
    answer = chat(question)
    print(f"💊 Amino AI Response: {answer}")
    print("-" * 50)
    test_results.append({"question": question, "answer": answer})

# question = "What's your name and what can you do?"
# print(f"\n🏥 Medical Question: {question}")
# answer = chat(question)
# print(f"💊 Amino AI Response: {answer}")
# print("-" * 50)


🏥 Medical Question 1: What are the symptoms of diabetes?
💊 Amino AI Response: Sure! Diabetes is a group of diseases characterized by high blood sugar levels (hyperglycemia). Here are some common symptoms of diabetes:

1. High blood sugar (hyperglycemia): This causes a range of symptoms including:
   - Blurred vision
   - Thirst
   - Hungry all the time
   - Fatigue
   - Dizziness
   - Poor concentration
   - Numbness or tingling in hands and feet
2. Sweating: In addition to the symptoms mentioned above, individuals with diabetes may also experience excessive sweating, which can be particularly uncomfortable during hot weather.
3. Urine: Individuals with diabetes may notice increased urination due to their higher than normal blood sugar levels.
4. Eye problems: Diabetes can cause damage to the retina, which can lead to blurry vision.
5. Leg swelling: Some people with diabetes may experience swelling in their legs or ankles, particularly at night when they wake up feeling dizzy.
6. Foot

In [ ]:
print(f"Loading test dataset...")
raw_test_dataset = load_from_disk(test_dataset_gdrive_path)

Loading test dataset...


In [ ]:
def parse_chat_string(example):
  text = example['text']
  try:
    # Split the text by the end-of-sentence token used in this template
    parts = text.split('</s>')
    # The user content is the part after the <|user|> tag
    user_content = parts[1].split('<|user|>\n')[1].strip()
    # The assistant content is the part after the <|assistant|> tag
    assistant_content = parts[2].split('<|assistant|>\n')[1].strip()
    return {'question': user_content, 'answer': assistant_content}
  except (IndexError, AttributeError):
    # Return empty strings if parsing fails for any entry
    return {'question': '', 'answer': ''}

print("Parsing questions and answers...")
test_dataset = raw_test_dataset.map(parse_chat_string, remove_columns=['text'])
# Filter out any examples that failed to parse
test_dataset = test_dataset.filter(lambda example: len(example['question']) > 0)

print(f"Successfully loaded and parsed {len(test_dataset)} examples.")

Parsing questions and answers...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Successfully loaded and parsed 1000 examples.


In [ ]:
predictions = []
references = [] # This is the "ground truth"
questions = []

print(f"\nGenerating predictions for {len(test_dataset)} test examples...")

for example in tqdm(test_dataset):
    question = example["question"]
    reference_answer = example["answer"]

    model_answer = chat(question)

    predictions.append(model_answer)
    references.append(reference_answer)
    questions.append(question)

print("\nAll predictions generated.")


Generating predictions for 1000 test examples...


100%|██████████| 1000/1000 [2:17:38<00:00,  8.26s/it]


All predictions generated.


In [ ]:
print("Storing test cases with generated answers...")
file_path = gdrive_path + "/test_cases_with_generated_answers.json"
with open(file_path, 'w') as f:
    json.dump({
        "predictions": predictions,
        "references": references,
        "questions": questions
    }, f, indent=4)

In [ ]:
print("\n" + "="*50)
print("QUANTITATIVE EVALUATION (AUTOMATED METRICS)")
print("="*50)
print("Calculating scores for the entire test set...")

# Calculate ROUGE Score
rouge_metric = evaluate.load('rouge')
rouge_results = rouge_metric.compute(predictions=predictions, references=references)

# Calculate BERTScore
bertscore_metric = evaluate.load("bertscore")
bert_results = bertscore_metric.compute(predictions=predictions, references=references, lang="en")

print("\n--- ROUGE SCORES ---")
print("Measures word/phrase overlap between prediction and ground truth.")
print(f"ROUGE-1 (Unigram Overlap): {rouge_results['rouge1']:.4f}")
print(f"ROUGE-2 (Bigram Overlap):  {rouge_results['rouge2']:.4f}")
print(f"ROUGE-L (Longest Common Subsequence): {rouge_results['rougeL']:.4f}")

print("\n--- BERTSCORE ---")
print("Measures semantic similarity between prediction and ground truth.")
# We calculate the average of the BERTScore lists
avg_precision = sum(bert_results['precision']) / len(bert_results['precision'])
avg_recall = sum(bert_results['recall']) / len(bert_results['recall'])
avg_f1 = sum(bert_results['f1']) / len(bert_results['f1'])
print(f"Precision: {avg_precision:.4f}")
print(f"Recall:    {avg_recall:.4f}")
print(f"F1 Score:  {avg_f1:.4f}")

print("\nEvaluation complete.")

print("\nStoring Evaluation results to Google Drive")
evaluation_metrics = {
    "rouge": {
        "rouge1": rouge_results['rouge1'],
        "rouge2": rouge_results['rouge2'],
        "rougeL": rouge_results['rougeL']
    },
    "bertscore": {
        "precision": avg_precision,
        "recall": avg_recall,
        "f1": avg_f1
    }
}

eval_file_path = gdrive_path + "/eval_results.json"
with open(eval_file_path, 'w') as f:
    json.dump(evaluation_metrics, f, indent=4)

print(f"Evaluation results have been saved to: {eval_file_path}")


QUANTITATIVE EVALUATION (AUTOMATED METRICS)
Calculating scores for the entire test set...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- ROUGE SCORES ---
Measures word/phrase overlap between prediction and ground truth.
ROUGE-1 (Unigram Overlap): 0.2752
ROUGE-2 (Bigram Overlap):  0.0647
ROUGE-L (Longest Common Subsequence): 0.1530

--- BERTSCORE ---
Measures semantic similarity between prediction and ground truth.
Precision: 0.8416
Recall:    0.8334
F1 Score:  0.8370

Evaluation complete.

Storing Evaluation results to Google Drive
Evaluation results have been saved to: /content/drive/My Drive/LLM-FineTuning/eval_results.json
